### 这个 是 检测版本，用少量的 公司 ticker 来检测 能否顺利 download 每一个 financial table

In [1]:
import requests
from bs4 import BeautifulSoup  # WEB SCRAPING
import pandas as pd            # Build table
import time              
from time import sleep         # avoid web site block
import yfinance as yf          # stock price
import numpy as np             # table calculation
from random import randint     # sleep random time prevent website block
pd.options.display.max_columns = None
pd.options.display.max_rows = None

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"}

这个code / function 可以 pull trailing 12 month 的 income statement 的 数据
如果想 改成 anual 应该也比较容易
其他的  balance sheet， cash flow， ratios 这些 table 需要重新 写 code，需要一些 微修改

思路： 首先 将 一整个 界面的 表格 爬下来； 然后 结合 之前 的 code （首先 订好 需要提取的 column names， 然后可以从 第一步生成 的 table 里直接 extract 信息）

In [ ]:
https://stockanalysis.com/stocks/wix/financials/trailing/
https://stockanalysis.com/stocks/wix/financials/balance-sheet/trailing/
https://stockanalysis.com/stocks/wix/financials/cash-flow-statement/trailing/
https://stockanalysis.com/stocks/wix/financials/ratios/trailing/

https://stockanalysis.com/stocks/wix/forecast/

In [ ]:
Ticker = ['ABNB','AI','APP','APPN','APPS','ASAN','AVLR','AYX','BILL','BIRD','BKNG','CAKE','CAR','CFLT','COUR','CRTO','DDOG'
,'DOCS','DOCU','DRI','DT','DUOL','ELY','ESTC','EXLS','EXPE','FIGS','FL','FORG','FRSH','FUN','GDDY','GOOS','GRPN'
,'HUBS','IHRT','JOUT','LULU','MDB','MGNI','MNDY','MTCH','MTN','NCNO','NET','NEWR','NKE','OKTA','ONON','PATH','PAYC'
,'PAYX','PD','PINS','PLAY','PLTR','PUBM','PYCR','QLYS','RNG','ROKU','RVLV','SBUX','SEAS','SFIX','SHOP','SIX','SMAR'
,'SNAP','SNOW','SONO','SPLK','SPOT','SPT','SQSP','SWBI','SWKS','TCOM','TDOC','TEAM','TPR','TTD'
,'TWLO','TWOU','TWTR','U','VEEV','VSTO','W','WDAY','WDC','WIX','YETI','YELP','UBER','LYFT','YOU','Z','ZEN','ZI','ZM']

In [61]:
Ticker = ['MSFT','NOW','TTD','AAPL', 'AMZN','CRM', 'ADBE', 'WDAY', 'SAP', 'INTU', 'PANW',
         'IBM', 'CSCO', 'FTNT', 'FVRR', 'ZM', 'DDOG', 'ADSK', 'TEAM', 'SNPS', 'CDNS',
         'PAYC', 'SPLK', 'HUBS', 'ZEN',  'ROP', 'ANSS'] 

In [62]:
# Get stock price

#ticker = ['MSFT', 'NOW','TTD','AAPL', 'DE','CRM']  

table_columns = ['Most_Recent_Stock_Price']
Company_Stock_Price = pd.DataFrame(columns=table_columns)

ticker_index = 0
num = 0

for i in range(len(Ticker)):
    data = yf.Ticker(Ticker[ticker_index]).info
    price = data['regularMarketPrice']
    #price_table.append(price)
    ticker_index = ticker_index + 1
    Company_Stock_Price.loc[num] = price
    num = num + 1

Company_Stock_Price.index = Ticker
Company_Stock_Price


,Most_Recent_Stock_Price
MSFT,265.23
NOW,435.78
TTD,62.52
AAPL,161.38
AMZN,129.79
CRM,160.21
ADBE,375.26
WDAY,160.89
SAP,84.58
INTU,436.80


In [12]:
def get_table(ticker):
    url = f'https://stockanalysis.com/stocks/{ticker}/financials/trailing/'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    raw_data = soup.find_all('tr') # this is the data source

    # Create a table for specific company to build its financial metrics
    # assign column names to this table

    column_num = [1,2,3,4,5] # given some company with limited financial information, only pick 5
    column_name = []
    for i in column_num:
        column = raw_data[0].find_all(class_='svelte-17fayh1')[i].string
        column_name.append(column)

    Trailing_Income_Statement = pd.DataFrame(columns=column_name)


    # get row names
    row_name = []
    for i in range(1, len(raw_data), 1):
        row = raw_data[i].find_all(class_='svelte-1d8dp52')[0].findAll('span')[0].string
        row_name.append(row)

    # get cell information row by row    
    r = 1
    num = 0
    j=0

    for r in range(1, len(raw_data), 1):
        row_cell = []
        for j in column_num:
            cell = raw_data[r].find_all(class_='svelte-1d8dp52')[j].string
            row_cell.append(cell)
        Trailing_Income_Statement.loc[num] = row_cell
        num = num + 1  

    Trailing_Income_Statement.index = row_name
    Trailing_Income_Statement = Trailing_Income_Statement.reset_index()
    
    return Trailing_Income_Statement

In [13]:
get_table('MSFT')

,index,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30
0,Revenue,"198,270","192,557","184,903","176,251","168,088"
1,Revenue Growth (YoY),17.96%,20.37%,20.63%,19.81%,17.53%
2,Cost of Revenue,"62,650","60,212","57,642","54,876","52,232"
3,Gross Profit,"135,620","132,345","127,261","121,375","115,856"
4,"Selling, General & Admin","27,725","27,051","26,385","25,708","25,224"
5,Research & Development,"24,512","23,350","22,248","21,389","20,716"
6,Other Operating Expenses,0,0,0,0,0
7,Operating Expenses,"52,237","50,401","48,633","47,097","45,940"
8,Operating Income,"83,383","81,944","78,628","74,278","69,916"
9,Interest Expense / Income,"2,063","2,120","2,250","2,296","2,346"


In [17]:
Ticker = ['MSFT','FROG','DUOL','WIX', 'GTLB']

T1_IS_columns = ['R_0','R_P1','R_P2','R_P3','R_P4', 
              'EBITDA_0','EBITDA_P1','EBITDA_P2','EBITDA_P3','EBITDA_P4',
              'EPS_0','EPS_P1','EPS_P2','EPS_P3','EPS_P4']
T1_IncomeStatement = pd.DataFrame(columns=T1_IS_columns)

T2_columns = ['Current_Quater']
T2_Time = pd.DataFrame(columns=T2_columns)



ticker_index = 0
num = 0

for ticker_index in range(len(Ticker)):
    Trailing_Income_Statement = get_table(Ticker[ticker_index])
    financials = ['Revenue','EBITDA','EPS (Diluted)']
    data = Trailing_Income_Statement[Trailing_Income_Statement['index'].isin(financials)]
    
    quarter_data = []
    quarter_data.append(data.columns[1])
    T2_Time.loc[num] = quarter_data
    
    financial_data = []
    financials_year_index = [ 1, 2, 3, 4, 5]    # trailing 12 months, started from most recent quarter

    for i in range(len(financials)):
        for j in financials_year_index:
            cell_data = data[data['index'] == financials[i]].iloc[0,j]
            financial_data.append(cell_data)
    
    T1_IncomeStatement.loc[num] = financial_data
    num = num + 1
    sleep(randint(1,9))    

T1_IncomeStatement.index = Ticker
T2_Time.index = Ticker

In [20]:
T1_IncomeStatement

,R_0,R_P1,R_P2,R_P3,R_P4,EBITDA_0,EBITDA_P1,EBITDA_P2,EBITDA_P3,EBITDA_P4,EPS_0,EPS_P1,EPS_P2,EPS_P3,EPS_P4
MSFT,"198,270","192,557","184,903","176,251","168,088","100,239","98,579","94,918","90,051","85,134",9.65,9.58,9.39,8.94,8.05
FROG,244.44,225.29,206.68,190.14,175.32,-63.44,-58.34,-52.77,-42.13,-30.02,-0.89,-0.79,-0.68,-0.46,-0.39
DUOL,306.22,276.63,250.77,226.03,207.74,-69.92,-55.73,-57.23,-50.36,-24.7,-1.95,-1.58,-2.57,-2.84,-2.11
WIX,"1,335.96","1,306.31","1,269.66","1,215.29","1,148.68",-336.76,-335.1,-306.07,-266.53,-248.47,-7.54,-5.02,-2.06,-0.44,-1.75
GTLB,-,252.65,-,196.36,-,-,-155.44,-,-212.29,-,-,-1.95,-,-4.23,-


In [21]:
T2_Time

,Current_Quater
MSFT,2022-06-30
FROG,2022-06-30
DUOL,2022-06-30
WIX,2022-06-30
GTLB,2022-04-30


In [22]:
def get_balance_sheet(ticker):
    url = f'https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/trailing/'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    raw_data = soup.find_all('tr') # this is the data source

    column_num = [1,2,3,4,5] # given some company with limited financial information, only pick 5
    column_name = []
    for i in column_num:
        column = raw_data[0].find_all(class_='svelte-17fayh1')[i].string
        column_name.append(column)

    Trailing_Balance_Sheet = pd.DataFrame(columns=column_name)

    # get row names
    row_name = []
    for i in range(1, len(raw_data), 1):
        row = raw_data[i].find_all(class_='svelte-1d8dp52')[0].findAll('span')[0].string
        row_name.append(row)

    # get cell information row by row    
    r = 1
    num = 0
    j=0

    for r in range(1, len(raw_data), 1):
        row_cell = []
        for j in column_num:
            cell = raw_data[r].find_all(class_='svelte-1d8dp52')[j].string
            row_cell.append(cell)
        Trailing_Balance_Sheet.loc[num] = row_cell
        num = num + 1  

    Trailing_Balance_Sheet.index = row_name
    Trailing_Balance_Sheet = Trailing_Balance_Sheet.reset_index()  
    
    return Trailing_Balance_Sheet

In [23]:
Ticker = ['MSFT','FROG','DUOL','WIX', 'GTLB']

T3_columns = ['Cash_0','Cash_P1','Cash_P2','Cash_P3','Cash_P4', 
              'CA_0','CA_P1','CA_P2','CA_P3','CA_P4',
              'CL_0','CL_P1','CL_P2','CL_P3','CL_P4',
             'LL_0','LL_P1','LL_P2','LL_P3','LL_P4']
T3_BalanceSheet = pd.DataFrame(columns=T3_columns)

#T2_columns = ['Current_Quater']
#T2 = pd.DataFrame(columns=T2_columns)



ticker_index = 0
num = 0

for ticker_index in range(len(Ticker)):
    Trailing_BalanceSheet = get_balance_sheet(Ticker[ticker_index])
    financials = ['Cash & Cash Equivalents','Total Current Assets','Total Current Liabilities','Total Long-Term Liabilities']
    data = Trailing_BalanceSheet[Trailing_BalanceSheet['index'].isin(financials)]
    
    #quarter_data = []
    #quarter_data.append(data.columns[1])
    #T2.loc[num] = quarter_data
    
    financial_data = []
    financials_year_index = [ 1, 2, 3, 4, 5]    # trailing 12 months, started from most recent quarter

    for i in range(len(financials)):
        for j in financials_year_index:
            cell_data = data[data['index'] == financials[i]].iloc[0,j]
            financial_data.append(cell_data)
    
    T3_BalanceSheet.loc[num] = financial_data
    num = num + 1
    sleep(randint(1,9))    

T3_BalanceSheet.index = Ticker
#T2.index = Ticker

In [24]:
T3_BalanceSheet

,Cash_0,Cash_P1,Cash_P2,Cash_P3,Cash_P4,CA_0,CA_P1,CA_P2,CA_P3,CA_P4,CL_0,CL_P1,CL_P2,CL_P3,CL_P4,LL_0,LL_P1,LL_P2,LL_P3,LL_P4
MSFT,"104,757","104,693","125,369","130,615","130,334","169,684","153,922","174,188","174,326","184,406","95,082","77,439","77,510","80,528","88,657","103,216","104,244","102,869","102,912","103,134"
FROG,430.15,427.65,421.13,402.38,615.23,510.19,507.35,499.02,469.75,668.54,193.73,182.76,175.26,147.17,134.56,38.52,37.12,38.68,38.27,32.3
DUOL,591.16,577.33,553.92,549.44,114.64,655.15,640.64,619.27,596.34,162.57,147.33,135.78,119.13,98.65,85.05,25.51,28.02,29.12,8.4,7.81
WIX,"1,173.81","1,206.64","1,326.57","1,252.59","1,286.1","1,248.67","1,303.92","1,389.81","1,316.95","1,352.82","1,196.54",823.62,774.3,720.36,711.4,851.46,"1,114.66","1,139.77","1,139.02","1,100.56"
GTLB,934.78,934.7,924.75,276.25,0,"1,044.23","1,051.84","1,012.35",350.71,-,234.6,241.6,178.89,151.46,-,46.15,50.57,40.51,40.5,-


In [25]:
def get_Cashflow(ticker):
    url = f'https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/trailing/'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    raw_data = soup.find_all('tr') # this is the data source

    column_num = [1,2,3,4,5] # given some company with limited financial information, only pick 5
    column_name = []
    for i in column_num:
        column = raw_data[0].find_all(class_='svelte-17fayh1')[i].string
        column_name.append(column)

    Trailing_Cashflow = pd.DataFrame(columns=column_name)

    # get row names
    row_name = []
    for i in range(1, len(raw_data), 1):
        row = raw_data[i].find_all(class_='svelte-1d8dp52')[0].findAll('span')[0].string
        row_name.append(row)

    # get cell information row by row    
    r = 1
    num = 0
    j=0

    for r in range(1, len(raw_data), 1):
        row_cell = []
        for j in column_num:
            cell = raw_data[r].find_all(class_='svelte-1d8dp52')[j].string
            row_cell.append(cell)
        Trailing_Cashflow.loc[num] = row_cell
        num = num + 1  

    Trailing_Cashflow.index = row_name
    Trailing_Cashflow = Trailing_Cashflow.reset_index()  
    
    return Trailing_Cashflow

In [26]:
Ticker = ['MSFT','FROG','DUOL','WIX', 'GTLB']

T4_columns = ['FreeCashflow_0','FreeCashflow_P1','FreeCashflow_P2','FreeCashflow_P3','FreeCashflow_P4']
T4_Cashflow = pd.DataFrame(columns=T4_columns)

#T2_columns = ['Current_Quater']
#T2 = pd.DataFrame(columns=T2_columns)



ticker_index = 0
num = 0

for ticker_index in range(len(Ticker)):
    Trailing_Cashflow = get_Cashflow(Ticker[ticker_index])
    financials = ['Free Cash Flow']
    data = Trailing_Cashflow[Trailing_Cashflow['index'].isin(financials)]
    
    #quarter_data = []
    #quarter_data.append(data.columns[1])
    #T2.loc[num] = quarter_data
    
    financial_data = []
    financials_year_index = [1, 2, 3, 4, 5]    # trailing 12 months, started from most recent quarter

    for i in range(len(financials)):
        for j in financials_year_index:
            cell_data = data[data['index'] == financials[i]].iloc[0,j]
            financial_data.append(cell_data)
    
    T4_Cashflow.loc[num] = financial_data
    num = num + 1
    sleep(randint(1,9))    

T4_Cashflow.index = Ticker
#T2.index = Ticker

In [27]:
T4_Cashflow

,FreeCashflow_0,FreeCashflow_P1,FreeCashflow_P2,FreeCashflow_P3,FreeCashflow_P4
MSFT,"65,149","63,649","60,693","60,420","56,118"
FROG,4.82,19.89,23.67,18.96,47.29
DUOL,29.95,17.8,2.96,5.99,3.42
WIX,-50.86,-20.24,27.99,43.82,71.9
GTLB,-,-53.36,-,-60.14,-


In [28]:
def get_MCEV(ticker):
    url = f'https://stockanalysis.com/stocks/{ticker}/financials/ratios/trailing/'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    raw_data = soup.find_all('tr') # this is the data source

    column_num = [1,2,3,4] # given some company with limited financial information, only pick 5
    column_name = []
    for i in column_num:
        column = raw_data[0].find_all(class_='svelte-17fayh1')[i].string
        column_name.append(column)

    Trailing_MCEV = pd.DataFrame(columns=column_name)

    # get row names
    row_name = []
    row_name.append('Market Capitalization') # due to slightly differnt text extraction, use hardcode instead
    for i in range(2, len(raw_data), 1):
        row = raw_data[i].find_all(class_='svelte-1d8dp52')[0].findAll('span')[0].string
        row_name.append(row)

    # get cell information row by row    
    r = 1
    num = 0
    j=0

    for r in range(1, len(raw_data), 1):
        row_cell = []
        for j in column_num:
            cell = raw_data[r].find_all(class_='svelte-1d8dp52')[j].string
            row_cell.append(cell)
        Trailing_MCEV.loc[num] = row_cell
        num = num + 1  

    Trailing_MCEV.index = row_name
    Trailing_MCEV = Trailing_MCEV.reset_index()   
    
    return Trailing_MCEV

In [29]:
Ticker = ['MSFT','FROG','DUOL','WIX', 'GTLB']

T5_columns = ['MC_Current','MC0', 'MC_P1', 'MC_P2', 'EV_Current','EV_0', 'EV_P1', 'EV_P2']
T5_MCEV = pd.DataFrame(columns=T5_columns)


ticker_index = 0
num = 0

for ticker_index in range(len(Ticker)):
    Trailing_MCEV = get_MCEV(Ticker[ticker_index])
    financials = ['Market Capitalization','Enterprise Value']
    data = Trailing_MCEV[Trailing_MCEV['index'].isin(financials)]
    
    #quarter_data = []
    #quarter_data.append(data.columns[1])
    #T2.loc[num] = quarter_data
    
    financial_data = []
    financials_year_index = [ 1, 2, 3, 4]    # trailing 12 months, started from most recent quarter

    for i in range(len(financials)):
        for j in financials_year_index:
            cell_data = data[data['index'] == financials[i]].iloc[0,j]
            financial_data.append(cell_data)
    
    T5_MCEV.loc[num] = financial_data
    num = num + 1
    sleep(randint(1,9))    

T5_MCEV.index = Ticker

In [30]:
T5_MCEV

,MC_Current,MC0,MC_P1,MC_P2,EV_Current,EV_0,EV_P1,EV_P2
MSFT,"1,976,937","1,920,840","2,311,359","2,525,084","1,933,450","1,877,353","2,267,949","2,463,747"
FROG,"2,126","2,082","2,625","2,863","1,721","1,677","2,222","2,469"
DUOL,"3,679","3,429","3,674","3,976","3,113","2,863","3,124","3,451"
WIX,"3,720","3,768","5,965","9,044","3,690","3,737","5,801","8,751"
GTLB,"9,277","7,074","9,269","16,046","8,342","6,140","8,334","15,122"


In [59]:
# Table 6: Future Revenue & EPS [Should able to be applied to most company]
Ticker = ['MSFT','FROG','DUOL','WIX', 'GTLB']


T6_columns = ['R_P1','R_0', 'R_F1', 'EPS_P1','EPS_0', 'EPS_F1']
T6_Forecast = pd.DataFrame(columns=T6_columns)


ticker_index = 0
num = 0

for ticker_index in range(len(Ticker)):
    url = f'https://stockanalysis.com/stocks/{Ticker[ticker_index]}/forecast/'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    raw_data = soup.find_all('tr') # this is the data source
    
    
    financial_data = []
    financials_index = [11, 13]               
    financials_year_index = [ -6, -5, -4]    

    for i in financials_index:            #extract Revenue, EBITDA
        for j in financials_year_index:   #extract title, data of 2021, 2020 & 2019
            cell_data = raw_data[i].find_all('td')[j].string
            financial_data.append(cell_data)
            
    T6_Forecast.loc[num] = financial_data
    #current_company_financials.index = ticker[ticker_index]
    num = num + 1
    #ticker_index = ticker_index + 1
    sleep(randint(1,9))

    
T6_Forecast.index = Ticker

T6_Forecast['R_P1'] = np.where(T6_Forecast['R_P1'].str.contains('B'),
                              pd.to_numeric(T6_Forecast['R_P1'].map(lambda x:str(x)[:-1])) *1000,
                              pd.to_numeric(T6_Forecast['R_P1'].map(lambda x:str(x)[:-1])) *1)
T6_Forecast['R_0'] = np.where(T6_Forecast['R_0'].str.contains('B'),
                              pd.to_numeric(T6_Forecast['R_0'].map(lambda x:str(x)[:-1])) *1000,
                              pd.to_numeric(T6_Forecast['R_0'].map(lambda x:str(x)[:-1])) *1)
T6_Forecast['R_F1'] = np.where(T6_Forecast['R_F1'].str.contains('B'),
                              pd.to_numeric(T6_Forecast['R_F1'].map(lambda x:str(x)[:-1])) *1000,
                              pd.to_numeric(T6_Forecast['R_F1'].map(lambda x:str(x)[:-1])) *1)


In [60]:
T6_Forecast

,R_P1,R_0,R_F1,EPS_P1,EPS_0,EPS_F1
MSFT,168090.00,198270.00,231680.00,8.12,9.70,10.96
FROG,206.68,282.62,360.21,-0.68,-0.00,0.08
DUOL,250.77,361.34,461.51,-2.57,-1.81,-1.58
WIX,1270.00,1440.00,1670.00,-2.06,-1.44,-0.36
GTLB,252.65,407.87,567.75,-1.95,-0.91,-0.79
